In [ ]:
import qiskit as qs
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from itertools import product

In [ ]:
from sympy.physics.wigner import gaunt
from sympy.physics.hydrogen import R_nl
from scipy.integrate import dblquad

#precompute the radial integrals
n_max = 2
rad_integrals = {}

nls = []
i = 0
for n in range(1, n_max+1):
    for l in range(0, n):
        nls.append((n,l))
nls_2 = []
for i in range(len(nls)):
    for j in range(i+1):
        nls_2.append([nls[i],nls[j]])

nls_4 = []
for i in range(len(nls_2)):
    for j in range(i+1):
        nls_4.append(nls_2[i]+nls_2[j])

for k in range(5*n_max):
    _rad_integrals = {}
    for i, nl in enumerate(nls_4):
        def f(r1,r2):
            return r1**2*r2**2*R_nl(nl[0][0],nl[0][1],r1)*R_nl(nl[1][0],nl[1][1],r2)*R_nl(nl[2][0],nl[2][1],r1)*R_nl(nl[3][0],nl[3][1],r2)*min(r1,r2)**k/max(r1,r2)**(k+1)    
        print(i)
        _rad_integrals[tuple(nl)] = dblquad(f, 0.1, 15, 0.1, 15)[0]
    rad_integrals[k] = _rad_integrals

import pickle
with open("integrals.pkl", "wb") as f:
    pickle.dump(rad_integrals, f)

def ordered_tuple(n, l):
    nl_tuple = ((ni,li) for ni,nl in zip(n,l))
    if nl_tuple[0][0] < nl_tuple[1][0] or (nl_tuple[0][0] == nl_tuple[1][0] and nl_tuple[0][1] < nl_tuple[1][1]):
        nl_tuple[0], nl_tuple[1] = nl_tuple[1], nl_tuple[0]
    if nl_tuple[2][0] < nl_tuple[3][0] or (nl_tuple[2][0] == nl_tuple[3][0] and nl_tuple[2][1] < nl_tuple[3][1]):
        nl_tuple[2], nl_tuple[3] = nl_tuple[2], nl_tuple[3]

    if (nl_tuple[0][0] < nl_tuple[2][0] or
        (nl_tuple[0][0] == nl_tuple[2][0] and nl_tuple[0][1] < nl_tuple[2][1]) or
        (nl_tuple[0][0] == nl_tuple[2][0] and nl_tuple[0][1] == nl_tuple[2][1] and nl_tuple[1][0] < nl_tuple[3][0]) or 
        (nl_tuple[0][0] == nl_tuple[2][0] and nl_tuple[0][1] == nl_tuple[2][1] and nl_tuple[1][0] == nl_tuple[3][0] and nl_tuple[1][1] < nl_tuple[3][1])):
            nl_tuple[0], nl_tuple[1], nl_tuple[2], nl_tuple[3] = nl_tuple[2], nl_tuple[3], nl_tuple[0], nl_tuple[1]

def Coulomb_matrix_element(n: list, l: list, m: list, s: list):
    if not (len(n) == 4 and len(l) == 4 and len(m) == 4):
        raise Exception("n, l and m must be of length 4")

    if not (s[0] == s[2] and s[1] == s[3]):
        return 0

    mat_el = 0
    for k in range(5*max(n)):
        ang_factor = 1
        for p in range(-k, k+1):
            ang_factor += (4*np.pi/(2*k+1)*(-1)**(m[0]+m[1]+p)
                            *float(gaunt(l[0],l[2],k,-m[0],m[2],-p))
                            *float(gaunt(l[1],l[3],k,-m[1],m[3],p)))
        if np.abs(ang_factor) > 1e-6:
            def f(r1,r2):
                return r1**2*r2**2*R_nl(n[0],l[0],r1)*R_nl(n[1],l[1],r2)*R_nl(n[2],l[2],r1)*R_nl(n[3],l[3],r2)*min(r1,r2)**k/max(r1,r2)**(k+1)
            rad_factor = dblquad(f, 0.1, 15, 0.1, 15)[0]
            mat_el += ang_factor*rad_factor
    return mat_el

In [ ]:
def orbital_energy(n_shells):
    n_qubits = int(n_shells*(n_shells+1)*(2*n_shells+1)/3)
    qubit_counter = 0
    pauli_list = []
    for n in range(1, n_shells+1):
        num_orbitals = 2*n**2
        for i in range(qubit_counter, qubit_counter+num_orbitals):
            pauli_list += [("Z", [i], -0.5/n**2), ("I", [i], -0.5/n**2)]
        qubit_counter += num_orbitals
    return SparsePauliOp.from_sparse_list(pauli_list, n_qubits).simplify()
        
def interaction_energy(n_shells):
    n_qubits = int(n_shells*(n_shells+1)*(2*n_shells+1)/3)
    quantum_numbers = []
    i = 0
    for n in range(1, n_shells+1):
        for l in range(n):
            for m in range(-l, l+1):
                for s in range(2):
                    quantum_numbers.append([i, (n,l,m,s-1/2)])
                    i += 1

    pauliop_tot = SparsePauliOp(n_qubits*"I", 0)
    for qn1, qn2, qn3, qn4 in product(quantum_numbers, repeat=4):
        mat_el = Coulomb_matrix_element([qn1[1][0], qn2[1][0], qn3[1][0], qn4[1][0]],
                                        [qn1[1][1], qn2[1][1], qn3[1][1], qn4[1][1]],
                                        [qn1[1][2], qn2[1][2], qn3[1][2], qn4[1][2]],
                                        [qn1[1][3], qn2[1][3], qn3[1][3], qn4[1][3]])

        if qn1[0] == qn2[0] or qn3[0] == qn4[0]:
            continue

        indices = [qn1[0], qn2[0], qn3[0], qn4[0]]
        pauliop = SparsePauliOp(n_qubits*"I")
        for i in range(4):
            sign = -1 if i < 2 else 1
            pauliop = pauliop.dot(SparsePauliOp.from_sparse_list(
                [(indices[i]*"Z"+"X", list(range(indices[i]+1)), 0.5),
                 (indices[i]*"Z"+"Y", list(range(indices[i]+1)), sign*0.5j)],
                n_qubits
            ))
        pauliop_tot += mat_el*pauliop.simplify()

    return -0.5*pauliop_tot.simplify()
        

        
def Hamiltonian(n_shells):
    return orbital_energy + 2*interaction_energy(n_shells)

In [ ]:
H = Hamiltonian(2)